# Handling a poorly conditioned covariance matrix

Sometimes, you may end up with a very clearly incorrect result from `kinisi`. 
For example, consider the MSD plot below. 

In [ ]:
import os
import numpy as np
import scipp as sc
import kinisi
from kinisi.analyze import DiffusionAnalyzer
from ase.io import read
from MDAnalysis import Universe
import matplotlib.pyplot as plt

credible_intervals = [[16, 84], [2.5, 97.5], [0.15, 99.85]]
alpha = [0.6, 0.4, 0.2]

path_to_file = os.path.join(os.path.dirname(kinisi.__file__), 
                            'tests/inputs/LiPS.exyz')
atoms = read(path_to_file, format='extxyz', index=':')

cell_dimensions = []
for frame in atoms:
    lengths = frame.cell.lengths()
    angles = frame.cell.angles()
    cell = [*lengths, *angles]
    cell_dimensions.append(cell)

u = Universe(path_to_file, 
             path_to_file, 
             format='XYZ', 
             topology_format='XYZ', 
             dt=20.0/1000)
for ts, dims in zip(u.trajectory, cell_dimensions):
    ts.dimensions = dims

params = {'specie': 'LI',
          'time_step': 0.001 * sc.Unit('ps'),
          'step_skip': 20 * sc.units.dimensionless,
          'progress': False
          }

diff = DiffusionAnalyzer.from_universe(u, **params)
diff.diffusion(1.5 * sc.Unit('ps'))

fig, ax = plt.subplots()
ax.plot(diff.dt.values, diff.msd.values, 'k-')
for i, ci in enumerate(credible_intervals):
    ax.fill_between(diff.dt.values,
                      *np.percentile(diff.distributions, ci, axis=1),
                      alpha=alpha[i],
                      color='#0173B2',
                      lw=0)
ax.set_xlabel(f'Time / {diff.dt.unit}')
ax.set_ylabel(f'MSD / {diff.msd.unit}')
ax.set_xlim(0, None)
ax.set_ylim(0, None)
plt.show()

Obviously something has gone wrong in this analysis. 
Unfortunately, this issue arises due to the numerical precision of computers, and there really isn't much that can be done to stop it. 
However, we can work around this issue. 

This problem arises due to the [condition number](https://en.wikipedia.org/wiki/Condition_number) of the covariance matrix being very high.

In [ ]:
np.linalg.cond(diff.diff.covariance_matrix.values)

This means that when the matrix is inverted, the result is very sensitive to small changes, on the scale of machine precision. 

`kinisi` tries its best to make sure that the matrix has a low enough condition number that this doesn't happen. 
But unfortunately, it is not always possible to achieve a low enough condition number. 
Therefore, we need to find a different solution. 

So far, the best solution that we have found has been to change the time intervals, `dt`, that is used.
Because we estimate the full covariance matrix, within reason, computing the MSD at every possible time interval is usually overkill. 
Instead, we can use a longer time interval sets, currently the time intervals are 0.02 ps apart. 

In [ ]:
diff.dt

We will reduce that so there is 0.1 ps between the time intervals, which we can do by setting a custom set of time intervals.  

In [ ]:
new_dt = sc.arange(dim='time interval', start=0.1 * sc.Unit('ps'),
                   stop=4.1 * sc.Unit('ps'),
                   step=0.1 * sc.Unit('ps'))

The new `dt` must be a subset of the old `dt` values, the cell below checks this. 

In [ ]:
print(np.all(np.any(np.isclose(new_dt.values[:, None], 
                               diff.dt.values[None, :]), 
                    axis=1)))

params['dt'] = new_dt

Running this through again, we can see that we get a much more reasonable value. 

In [ ]:
diff = DiffusionAnalyzer.from_universe(u, **params)
diff.diffusion(1.5 * sc.Unit('ps'))

fig, ax = plt.subplots()
ax.plot(diff.dt.values, diff.msd.values, 'k-')
for i, ci in enumerate(credible_intervals):
    ax.fill_between(diff.dt.values,
                      *np.percentile(diff.distributions, ci, axis=1),
                      alpha=alpha[i],
                      color='#0173B2',
                      lw=0)
ax.set_xlabel(f'Time / {diff.dt.unit}')
ax.set_ylabel(f'MSD / {diff.msd.unit}')
ax.set_xlim(0, None)
ax.set_ylim(0, None)
plt.show()

We welcome any user feedback on this problem, i.e., if you have a better why to mitigate it or remove it completely.  